<a href="https://colab.research.google.com/github/rajeevak40/Retail_Analytics/blob/main/Customer_Predictive_Analytics_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.linear_model import LogisticRegression

import pickle

# Data Preparation

In [3]:
# Loading data
df_purchase=pd.read_csv('/content/drive/MyDrive/Retail/purchase data.csv', index_col=0)

# importing pickel files
scaler= pickle.load(open('/content/drive/MyDrive/Retail/scaler.pickle','rb'))
pca= pickle.load(open('/content/drive/MyDrive/Retail/pca.pickle','rb'))
kmean_pca= pickle.load(open('/content/drive/MyDrive/Retail/kmean_pca.pickle','rb'))

# Column Filtering
df_features= df_purchase[['Sex', 'Marital status', 'Age','Education', 'Income', 'Occupation', 'Settlement size']]

# Scaling Data
df_purchase_segm_std=scaler.transform(df_features)


# PCA fitting
df_purchase_segm_pca=pca.transform(df_purchase_segm_std)

# KMeans Clustering
purchase_segm_pca_kmean = kmean_pca.predict(df_purchase_segm_pca)

# Create Purchase Predictor dataset 
df_purchase_predictors= df_purchase.copy()
df_purchase_predictors['Segment']= purchase_segm_pca_kmean

segment_dummies = pd.get_dummies(purchase_segm_pca_kmean, prefix='Segment', prefix_sep='_')
df_purchase_predictors= df_purchase_predictors.reset_index()
df_purchase_predictors=pd.concat([df_purchase_predictors, segment_dummies], axis=1)


In [4]:
df_purchase_predictors.head()


,ID,Day,Incidence,Brand,Quantity,Last_Inc_Brand,Last_Inc_Quantity,Price_1,Price_2,Price_3,Price_4,Price_5,Promotion_1,Promotion_2,Promotion_3,Promotion_4,Promotion_5,Sex,Marital status,Age,Education,Income,Occupation,Settlement size,Segment,Segment_0,Segment_1,Segment_2,Segment_3
0,200000001,1,0,0,0,0,0,1.59,1.87,2.01,2.09,2.66,0,1,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
1,200000001,11,0,0,0,0,0,1.51,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
2,200000001,12,0,0,0,0,0,1.51,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
3,200000001,16,0,0,0,0,0,1.52,1.89,1.98,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0
4,200000001,18,0,0,0,0,0,1.52,1.89,1.99,2.09,2.66,0,0,0,0,0,0,0,47,1,110866,1,0,2,0,0,1,0


# Purchase Probability Modeling
Check the impect of price in puchase decision and compute the magnitude of impact of price on price. 

Use of Logistic Regression will be better choice in this case.


In [5]:
Y=df_purchase_predictors['Incidence']
x=pd.DataFrame()
x['Mean']=(df_purchase_predictors['Price_1']+df_purchase_predictors['Price_2']+df_purchase_predictors['Price_3']+df_purchase_predictors['Price_4']+df_purchase_predictors['Price_5'])/5
model_purchase = LogisticRegression()
model_purchase.fit(x,Y)

LogisticRegression()

In [6]:
model_purchase.coef_

array([[-2.3480548]])

# Price Elasticity of Purchase Probability

In [8]:
#Check on the price range
df_purchase_predictors[['Price_1','Price_2','Price_3','Price_4','Price_5']].describe()

,Price_1,Price_2,Price_3,Price_4,Price_5
count,58693.000000,58693.000000,58693.000000,58693.000000,58693.000000
mean,1.392074,1.780999,2.006789,2.159945,2.654798
std,0.091139,0.170868,0.046867,0.089825,0.098272
min,1.100000,1.260000,1.870000,1.760000,2.110000
25%,1.340000,1.580000,1.970000,2.120000,2.630000
50%,1.390000,1.880000,2.010000,2.170000,2.670000
75%,1.470000,1.890000,2.060000,2.240000,2.700000
max,1.590000,1.900000,2.140000,2.260000,2.800000


In [13]:
price_range= pd.DataFrame(np.arange(0.5,3.5,0.01))
price_range

,0
0,0.50
1,0.51
2,0.52
3,0.53
4,0.54
...,...
295,3.45
296,3.46
297,3.47
298,3.48


In [15]:
y_predict= model_purchase.predict_proba(price_range)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  "X does not have valid feature names, but"
